In [17]:
## data processing
import os
import numpy as np
import pandas as pd

# graph
import plotly.express as px
import plotly.graph_objects as go

# web 개발 서버 flask 
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State

df = pd.read_csv("iris.csv")

#1. application 선언
app = dash.Dash() 
#2. layout Design
app.layout = html.Div(
    [
        dcc.Dropdown(id="input-col",
                    options=[
                        {"label":i,"value":i} for i in df.columns[:-1] 
                    ]),
        dcc.RadioItems(id="input-radio",
                      options = [
                          {"label":"평균","value":1},
                          {"label":"표준편차","value":2},
                          {"label":"합계","value":3},
                          {"label":"최대","value":4},
                          {"label":"최소","value":5},
                      ]),
        dcc.Graph(id="graph-figure"), 
    ]
)
## 집계함수를 선택할 수 있도록 평균, 표준편차, 합계, 최대, 최소 
@app.callback(
    Output("graph-figure","figure"),
    Input("input-col","value"),
    Input("input-radio","value")
)
def update_graph(col_name,aggr):
    if aggr == 1:
        dt = df.groupby(["Species"]).mean().loc[:,col_name]
        agg = "평균"
    elif aggr == 2:
        dt = df.groupby(["Species"]).std().loc[:,col_name]
        agg = "표준편차"
    elif aggr == 3:
        dt = df.groupby(["Species"]).sum().loc[:,col_name]
        agg = "합계"
    elif aggr == 3:
        dt = df.groupby(["Species"]).max().loc[:,col_name]
        agg = "최대"
    else:
        dt = df.groupby(["Species"]).min().loc[:,col_name]
        agg = "최소"
    title = f"{col_name}의 {agg}" 
    fig = go.Figure([go.Bar(x=dt.index, y=dt)])
    fig.update_layout(title_text=title)
    return fig


app.run_server(port=7777)